In [8]:
!pip install pillow pytesseract


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Attempting uninstall: packaging
    Found existing installation: packaging 21.0
    Uninstalling packaging-21.0:
      Successfully uninstalled packaging-21.0


In [3]:
from PyPDF2 import PdfReader
from PIL import Image
import pytesseract
import os, io



In [6]:
# 把pdf中的图片转成中文文字【效果不好，用gpt完善一下】
def extract_text_from_scanned_pdf(pdf_path, output_txt_path):
    # 初始化 PDF Reader
    reader = PdfReader(pdf_path)
    
    # 打开输出文件
    with open(output_txt_path, 'w', encoding='utf-8') as output_file:
        # 遍历每页
        for page_num, page in enumerate(reader.pages, start=1):
            # 获取页面的资源字典
            resources = page.get('/Resources')
            if not resources:
                print(f"Page {page_num} has no resources, skipping.")
                continue
            
            # 解引用资源字典
            resources = resources.get_object()
            xObject = resources.get('/XObject', {})
            if not xObject:
                print(f"Page {page_num} has no images, skipping OCR.")
                continue
            
            # 遍历 XObject 提取图片
            for obj_name, obj_ref in xObject.items():
                # 解引用图片对象
                obj = obj_ref.get_object()
                print(obj)
                if obj.get('/Subtype') == '/Image':
                    # 提取图片数据
                    width = obj.get('/Width')
                    height = obj.get('/Height')
                    data = obj.get_data()
                    
                    # 处理图片格式
                    # mode = "RGB" if obj.get('/ColorSpace') == '/DeviceRGB' else "P"
                    # img = Image.frombytes(mode, (width, height), data)
                    img = Image.open(io.BytesIO(data))
                    
                    # OCR 识别图片中的文字
                    text = pytesseract.image_to_string(img, lang='chi_sim')
                    output_file.write(f"--- Page {page_num} ---\n{text}\n")
                    print(f"Processed image from page {page_num}")
    
    print(f"OCR 处理完成，结果已保存至：{output_txt_path}")



# 输入 PDF 文件路径
pdf_path = "../data/被讨厌的勇气.pdf"  # 替换为你的 PDF 文件路径
output_txt_path = "../data/output_ocr.txt"  # 输出的文本路径

extract_text_from_scanned_pdf(pdf_path, output_txt_path)


{'/Type': '/XObject', '/Subtype': '/Image', '/Width': 1888, '/Height': 2764, '/ColorSpace': '/DeviceRGB', '/BitsPerComponent': 8, '/Filter': '/DCTDecode'}
Processed image from page 16
{'/Type': '/XObject', '/Subtype': '/Image', '/Width': 1888, '/Height': 2764, '/ColorSpace': '/DeviceGray', '/BitsPerComponent': 8, '/Filter': '/DCTDecode'}
Processed image from page 17
{'/Type': '/XObject', '/Subtype': '/Image', '/Width': 1888, '/Height': 2764, '/ColorSpace': '/DeviceGray', '/BitsPerComponent': 8, '/Filter': '/DCTDecode'}
Processed image from page 18
{'/Type': '/XObject', '/Subtype': '/Image', '/Width': 1888, '/Height': 2764, '/ColorSpace': '/DeviceGray', '/BitsPerComponent': 8, '/Filter': '/DCTDecode'}
Processed image from page 19
{'/Type': '/XObject', '/Subtype': '/Image', '/Width': 1888, '/Height': 2764, '/ColorSpace': '/DeviceGray', '/BitsPerComponent': 8, '/Filter': '/DCTDecode'}
Processed image from page 20
{'/Type': '/XObject', '/Subtype': '/Image', '/Width': 1888, '/Height': 2764,

In [20]:
# 清洗数据
import re

def clean_text(file_path):
    # 保留以“青年：”或“哲人：”开头的段落
    pattern = r'(青 年 :|哲 人:)'
    
    cleaned_lines = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            if line.startswith("--- Page "): # 过滤Page行
                continue
            if re.match(pattern, line.strip()):
                # 如果行以“青年：”或“哲人：”开头，保留换行
                cleaned_lines.append(line.strip())
            else:
                # 如果不是，去掉换行符，连接到前面的内容
                if cleaned_lines:
                    cleaned_lines[-1] += " " + line.strip()
                else:
                    cleaned_lines.append(line.strip())
    
    # 重新组合文本
    cleaned_text = "\n".join(cleaned_lines)
    return cleaned_text

# 清理文本
cleaned_text = clean_text(output_txt_path)
print(cleaned_text)




青 年 : 那 么 , 我 就 重 新 向 您 发 问 了 。 先 生 主 张 世 界 极 其 简 单 , 对 吧 ? 
哲 人: 是 的 。 世 界 简 单 得 令 人 难 以 置 信 , 人 生 也 是 一 样 。 
青 年 : 您 这 种 主 张 是 基 于 现 实 而 并 非 仅 仅 是 理 想 论 吗 ? 也 就 是 说 , 您 认 为 横 亘 在 你 我 人 生 中 的 种 种 问 题 也 是 简 单 的 吗 ? 
哲 人: 当 然 。 
青 年 : 好 吧 。 在 开 始 辩 论 之 前 , 请 允 许 我 先 说 明 一 下 此 次 造 访 的 目 的 。 首 先 , 我 冒 昧 造 访 的 首 要 缘 故 就 是 要 和 先 生 充 分 辩 论 , 以 见 分 晓 ; 其 次 , 如 果 可 能 的 话 , 我 希 望 能 让 先 生 您 收 回 自 己 的 主 张 。 
青 年 : 久 闻 先 生 大 名 。 据 说 此 地 住 着 一 位 与 众 不 同 的 哲 人 , 提 倡 不 容 小 麂 的 理 想 论 一 一 人 可 以 改 变 、 世 界 极 其 简 单 、 人 人 能 获 得 幸 福 。 对 我 来 说 , 先 生 的 这 些 论 调 我 都 无 法 接 受 。  所 以 , 我 想 用 自 己 的 眼 睛 去 确 认 , 哪 怕 是 微 小 的 不 当 之 处 也 要 给 您 纠 正 过 来 。 不 知 是 否 打 搅 您 了 ? 
哲 人: 没 有 , 欢 迎 之 至 。 我 自 己 也 正 期 待 着 倾 听 像 你 这 样 的 年 轻 人 的 心 声 以 丰 富 学 问 呢 。  
青 年 : 非 常 感 谢 。 其 实 我 也 并 非 是 想 要 不 分 青 红 皂 白 地 否 定 先 生 。 首 先 , 假 定 先 生 的 说 法 成 立 , 我 们 从 这 种 可 能 性 开 始 思 考 。  世 界 是 简 单 的 , 人 生 也 是 如 此 。 假 若 这 种 命 题 中 含 有 几 分 真 理 , 那 也 是 对 于 孩 子 的 世 界 而 言 。 孩 子 的 世 界 没 有 劳 动 或 纳 税 之 类 的 现 实 义 务 , 他 们 每 天 都 在 父 母 或 社 会 的 呵 护 下 自 由 自 在 地 生 活

In [21]:
with open("../data/cleaned_text.txt", "w", encoding="utf-8") as file:
    file.write(cleaned_text)

用大语言模型（Kimi）对cleaned_text.txt内容进行修正，这一步无代码！！！

In [71]:
def extract_dialogues(file_path):
    current_act_dialogues = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip()  # 去除两端空格
            line = line.replace(" ", "")  # 去除所有空格
            if "：" in line: # 按：切分角色和对话内容
                role, content = line.split("：", 1)
                role = role.strip()
                content = content.strip()
                current_act_dialogues.append({"role": role, "content": content})

    return current_act_dialogues

cleaned_path = "../data/cleaned_text_gpt.txt"
dialogues = extract_dialogues(cleaned_path)
print(dialogues)

[{'role': '青年', 'content': '那么，我就重新向您发问了。先生主张世界极其简单，对吧？'}, {'role': '哲人', 'content': '是的。世界简单得令人难以置信，人生也是一样。'}, {'role': '青年', 'content': '您这种主张是基于现实而并非仅仅是理想论吗？也就是说，您认为横亘在你我人生中的种种问题也是简单的吗？'}, {'role': '哲人', 'content': '当然。'}, {'role': '青年', 'content': '好吧。在开始辩论之前，请允许我先说明一下此次造访的目的。首先，我冒昧造访的首要缘故就是要和先生充分辩论，以见分晓；其次，如果可能的话，我希望能让先生您收回自己的主张。久闻先生大名。据说此地住着一位与众不同的哲人，提倡不容小觑的理想论——人可以改变、世界极其简单、人人能获得幸福。对我来说，先生的这些论调我都无法接受。所以，我想用自己的眼睛去确认，哪怕是微小的不当之处也要给您纠正过来。不知是否打搅您了？'}, {'role': '哲人', 'content': '没有，欢迎之至。我自己也正期待着倾听像你这样的年轻人的心声以丰富学问呢。'}, {'role': '青年', 'content': '非常感谢。其实我也并非是想要不分青红皂白地否定先生。首先，假定先生的说法成立，我们从这种可能性开始思考。世界是简单的，人生也是如此。假若这种命题中含有几分真理，那也是对于孩子的世界而言。孩子的世界没有劳动或纳税之类的现实义务，他们每天都在父母或社会的呵护下自由自在地生活，未来充满无限希望，自己也似乎无所不能。孩子们的眼睛被遮盖了，不必去面对丑恶的现实。的确，孩子眼中的世界呈现出简单的姿态。但是，随着年龄的增长，世界便逐渐露出真面目。人们不得不接受“我只不过如此”之类的现实，原以为等候在人生路上的一切“可能”都会变成“不可能”。幸福的浪漫主义季节转瞬即逝，残酷的现实主义时代终将到来。'}, {'role': '哲人', 'content': '你的话的确很有趣。'}, {'role': '青年', 'content': '还不仅如此。人一旦长大，就会被复杂的人际关系所困扰，被诸多的责任所牵绊。工作、家庭或者社会责任，一切都是。当然，孩提时代无法理解的歧视、战争或阶级之类的各种社会问题也

In [76]:
len(dialogues), len(qingnian_zheren_dialogues)

(1381, 693)

In [73]:
def extract_dialogues_list(dialogues):
    result = []
    qingnian_zheren_dialogues = []
    for i, dialogue in enumerate(dialogues):
        if "青年" in dialogue["role"]:
            qingnian_zheren_dialogues.append(dialogue)
        elif "哲人" in dialogue["role"]:
            qingnian_zheren_dialogues.append(dialogue)
            result.append(qingnian_zheren_dialogues)
            qingnian_zheren_dialogues = []
    return result

qingnian_zheren_dialogues = extract_dialogues_list(dialogues)
print(qingnian_zheren_dialogues)

[[{'role': '青年', 'content': '那么，我就重新向您发问了。先生主张世界极其简单，对吧？'}, {'role': '哲人', 'content': '是的。世界简单得令人难以置信，人生也是一样。'}], [{'role': '青年', 'content': '您这种主张是基于现实而并非仅仅是理想论吗？也就是说，您认为横亘在你我人生中的种种问题也是简单的吗？'}, {'role': '哲人', 'content': '当然。'}], [{'role': '青年', 'content': '好吧。在开始辩论之前，请允许我先说明一下此次造访的目的。首先，我冒昧造访的首要缘故就是要和先生充分辩论，以见分晓；其次，如果可能的话，我希望能让先生您收回自己的主张。久闻先生大名。据说此地住着一位与众不同的哲人，提倡不容小觑的理想论——人可以改变、世界极其简单、人人能获得幸福。对我来说，先生的这些论调我都无法接受。所以，我想用自己的眼睛去确认，哪怕是微小的不当之处也要给您纠正过来。不知是否打搅您了？'}, {'role': '哲人', 'content': '没有，欢迎之至。我自己也正期待着倾听像你这样的年轻人的心声以丰富学问呢。'}], [{'role': '青年', 'content': '非常感谢。其实我也并非是想要不分青红皂白地否定先生。首先，假定先生的说法成立，我们从这种可能性开始思考。世界是简单的，人生也是如此。假若这种命题中含有几分真理，那也是对于孩子的世界而言。孩子的世界没有劳动或纳税之类的现实义务，他们每天都在父母或社会的呵护下自由自在地生活，未来充满无限希望，自己也似乎无所不能。孩子们的眼睛被遮盖了，不必去面对丑恶的现实。的确，孩子眼中的世界呈现出简单的姿态。但是，随着年龄的增长，世界便逐渐露出真面目。人们不得不接受“我只不过如此”之类的现实，原以为等候在人生路上的一切“可能”都会变成“不可能”。幸福的浪漫主义季节转瞬即逝，残酷的现实主义时代终将到来。'}, {'role': '哲人', 'content': '你的话的确很有趣。'}], [{'role': '青年', 'content': '还不仅如此。人一旦长大，就会被复杂的人际关系所困扰，被诸多的责任所牵绊。工作、家庭或者社会责任，一切都是。当然，孩提时代无法理解的歧视、战争或阶级之

In [75]:
# 整理为json文件
import json

def build_json_data(dialogues, json_file):
    result = []
    for i, dialogue in enumerate(dialogues):
        content = []
        for act in dialogues[i]:
            content.append(act['content'])
        sample = {
            "instruction": content[0],
            "input": "",
            "output": content[-1]
        }
        result.append(sample)

    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

build_json_data(qingnian_zheren_dialogues, '../data/theCourageToBeDisliked_dataset.json')